# Een clustermodel gebaseerd op het Great Outdoors Data Warehouse

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
import pyodbc

In [ ]:
#verbinding maken aan het database
DB = {'servername': 'ENVY-AKRAM\\SQLEXPRESS','database': 'Great Outdoors - Akram en Sylvan'}
conn_str = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_conn = pyodbc.connect(conn_str, timeout=120)
export_cursor = export_conn.cursor()

engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

print(export_cursor.execute("SELECT @@version;"))
export_cursor

Dataset opzetten

In [ ]:
#inlezen
sales_staff = pd.read_sql_query("SELECT * FROM SALES_STAFF", engine)
sales_staff

In [ ]:
#juiste variabelen pakken
df = sales_staff.groupby('SALES_BRANCH_CODE').agg({
    'SALES_STAFF_CODE': 'count',
    'SALES_BRANCH_REGION': 'first',
    'SALES_BRANCH_POSTAL_ZONE': 'first',
    'COUNTRY_COUNTRY': 'first',
    'COUNTRY_CURRENCY_NAME': 'first'
    }).reset_index()
df

In [ ]:
#dummies maken
df_dummies = pd.get_dummies(df, columns=['COUNTRY_COUNTRY', 'COUNTRY_CURRENCY_NAME', 'SALES_BRANCH_REGION', 'SALES_BRANCH_POSTAL_ZONE'])
df_dummies

In [ ]:
#dummies concatenaten
df = pd.concat([df, df_dummies], axis=1)
df

In [ ]:
#drop de oude kolommen
df = df.drop(columns=['COUNTRY_COUNTRY', 'COUNTRY_CURRENCY_NAME', 'SALES_BRANCH_REGION', 'SALES_BRANCH_POSTAL_ZONE'])
df